# Extrayendo shapes

Algo que probablemente te resulte obvio a estas alturas es que producir datos en RDF con una calidad adecuada requiere cierto esfuerzo de modelado y de mantenimiento. RDF, como propuesta tecnológica, no fuerza la estructura de los datos en ningún caso, pero si los datos no tienen estructuras homogéneas y contstantes, nuestros grafos RDF se vuelven inoperables, pues se vuelve extremadamente difícil producir consultas SPARQL apropiadas para obtener la información deseada. Estas estructuras deben ser planificadas cuidadosamente, tanto a nivel topológico como semántico. Esto es, hemos de tener e implementar un plan que nos indique qué tipo de entidades deberían relacionarse con qué otro tipo de entidades, y cuáles son las propiedades y clases específicas que describen y relacionan tales entidades.

Los lenguajes de shapes (ShEx/SHACL) son una ayuda inestimable en ese sentido. Sirven tanto para describir esas estructuras como para detectar partes concretas del grafo donde las estructuras esperadas no se dan. Ambas cosas facilitan el mantenimiento y el aprovechamiento de los datos. No obstante, los esquemas ShEx/SHACL requieren un nuevo esfuerzo: hay que conocer bien el dominio para diseñarlos correctamente, hay que tener habilidad suficiente con la tecnología para escribirlos/utilizarlos, y hay que reescribirlos de forma paralela a al grafo en caso de que nuestro dominio/datos evolucionen. Es decir, si bien su utiidad es obvia, no podemos obviar que también es obvio que requieren aún un esfuerzo extra de mantenimiento que se añade al de mantener el propio grafo.


Para ayudar en esa tarea, aparece software especializado en la creación automática de shapes a partir de contenido RDF existente. Podemos distinguir dos tipos de aproximaciones, no necesariamente disjuntas.

Por un lado, algunas técnicas tratan de inferir shapes o mapear constraints presentes en conocimiento ontológico (T-BOX). Ejemplo: si se conoce que el rango de la propiedad vive_en es la clase Persona y el rango es la clase Vivienda, se podría inferir que la shapa asociada a la clase Persona tendrá una constraint que indica que se debe usar la propiedad vive_en para enlazar con nodos que conformen con la shape asociada a la clase vivienda. Sobre el papel, es una aproximación pulcra y con sentido, pero a menudo resulta insuficiente. Los usos concretos que cierto grafo de conocimiento hacen de una ontología suelen ser más específicos que las posibilidades de la ontología.

Por otro lado, hay técnicas que trabajan a nivel de conocimiento de instancia (A-BOX). Minan el vecindario de nodos de ejemplo y tratan de extrapolar conclusiones generales de lo que observan. Es decir, no utilizan conocimiento ontológico sobre *cómo deberían ser el grafo*, sino cnocimiento empírico sobre *cómo se observa que es el grafo*. Por ejemplo: si se observa que todos los nodos de tipo persona (Ana, Jose, Juan, Laura...) se enlazan mediante la propiedad vive_en con exactamente un nodo que es de tipo Vivienda (URIs representando dónde viven personas concretas), entonces se podrá asumir que mi grafo se describe correctamente con una shape shape_Persona que apunta mediante la propiedad vive_en a nodos que conforman con la shape shape_Vivienda. De nuevo, sobre el papel, parece un enfoque efectivo. No osbtante, los resultados pueden tender a ser simplistas cuando se dan relaciones complejas entre datos complejas (con operadores lógicos), podría perderse parte de la riquieza del dominio si los datos observados no contienen todos los posibles casos, y requieren más tiempo de computo que las aproximaciones que utilizan conocimiento T-BOX, puesto que en escenarios reales suele haber muchos más datos anivel de instancia que a nivel de clase.

Con todo, y a pesar que ninguna aproximación (ni la teórica combinación de ambas) asegura un resultado perfecto, este tipo de software facilita mucho la tarea de mantener esquemas de shapes.

A continuación, se describe cómo realizar extracciones de shapes con la librería de Python más madura a día de hoy para esta tarea: sheXer. sheXer es un extractor de shapes basado en A-BOX, es decir, extrae conclusiones sobre hechos observados a nivel de instancia. La libreria ofrece una clase (Shaper) con un método (shex_graph) para extraer shapes, pero los parámetros usados en la construcción de esta clase permiten adpaptarse a diferentes contextos. Antes de utilizar sheXer, debes tener claras tus intenciones con respecto a varias cuestiones:

* Tipo de entrada: fichero(s) local(es), ficheros remotos, grafo de rdflib, contenido expuesto en endpoint, contenido comprimido o no.
* Tipo de salida a nivel de lenguaje: ShEx (sintaxis compacta ShExC) o SHACL (sintaxis turtle).
* Tipo de salida a nivel de formato: salida a fichero, string, o JPG (como UML).
* Agrupación de nodos objetivo. Comúnmente, cuáles son las clases de las que se quiere obtener una shape. Pero también se pueden hacer asociaciones de grupos de nodos específicos con una shape mediante el uso de shape maps.

sheXer permite configurar muchos otros aspectos de la extracción, pero probablemente sean cuestiones que escapen a tus casos de uso iniciales. Consulta el repositorio de la librería para más información: https://github.com/weso/shexer

En el repositorio también encontrarás un Jupyter con código de ejemplo para usar sheXer en diferentes situaciones contra el endpoint de Wikidata.

A continuación se proporciona código de ejemplo que podrías adaptar a tus casos de uso:






In [2]:
!git clone https://github.com/cursosLabra/miw_websem2425.git

Cloning into 'miw_websem2425'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 49 (delta 21), reused 35 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (49/49), 61.63 KiB | 530.00 KiB/s, done.
Resolving deltas: 100% (21/21), done.


In [1]:
!pip install shexer

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.2/169.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.9/564.9 kB 22.2 MB/s eta 0:00:00
  Created wheel for wlighter: filename=wlighter-1.0.1-py3-none-any.whl size=5840 sha256=c7e1bcafd8932ce74f7a2270d04ef6af85f7c805cc7bcc77d2c1f8d9d05cc064
  Stored in directory: /root/.cache/pip/wheels/a7/99/fc/827abe031e1eb32a25e3bda1affff5a8adccfecf52d5548425
Successfully built wlighter


In [6]:
from shexer.shaper import Shaper
from shexer.consts import TURTLE_ITER

rdf_local_path = "/content/miw_websem2425/lab_sessions/books_data.ttl"

# Extracción de una shape asociada a cada clase con al menos alguna instancia en el grafo.

shaper = Shaper(graph_file_input=rdf_local_path,  # ruta local del grafo
                input_format=TURTLE_ITER,  # Formato de entrada TURTLE, pero con un parser no basado en rdflib.
                                           # const.TURTLE usa el parser de rdflib. La diferencia es que TURTLE_ITER
                                           # permite manejar grandes cargas de datos y no introduce espacios de nombres
                                           # por defecto.
                all_classes_mode=True, # Indica que queremos una shape por clase
                disable_comments=True) # Desactiva comentarios asociados a las shapes. Si no se indica esto, el esquema
                                       # de salida incluye anotaciones estadísticas.

result = shaper.shex_graph(string_output=True)  # Indica salida string. Hay parámetros para indicar salida UML o a fichero.

print(result)

PREFIX : <http://weso.es/shapes/>

:Person
{
   <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>  [<http://schema.org/Person>]  ;
   <http://xmlns.com/foaf/0.1/name>  <http://www.w3.org/2001/XMLSchema#string>  ;
   <http://schema.org/birthDate>  <http://www.w3.org/2001/XMLSchema#date>  ;
   <http://schema.org/nationality>  <http://www.w3.org/2001/XMLSchema#string>  ;
   <http://schema.org/knows>  @:Person  ?
}


:Genre
{
   <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>  [<http://schema.org/Genre>]  ;
   <http://schema.org/name>  <http://www.w3.org/2001/XMLSchema#string>  
}


:Book
{
   <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>  [<http://schema.org/Book>]  ;
   <http://purl.org/dc/terms/title>  <http://www.w3.org/2001/XMLSchema#string>  ;
   <http://schema.org/author>  @:Person  ;
   <http://schema.org/datePublished>  <http://www.w3.org/2001/XMLSchema#date>  ;
   <http://schema.org/genre>  @:Genre  +;
   <http://schema.org/ISBN>  <http://www.w3.org/2001/XMLSchema#string>  

In [12]:
from shexer.shaper import Shaper
from shexer.consts import TURTLE_ITER

rdf_local_path = "/content/miw_websem2425/lab_sessions/books_data.ttl"

namespaces_dict = {"http://www.w3.org/1999/02/22-rdf-syntax-ns#": "rdf",
                   "http://example.org/": "ex",
                   "http://www.w3.org/2001/XMLSchema#": "xsd",
                   "http://schema.org/" : "schema",
                   "http://xmlns.com/foaf/0.1/" : "foaf",
                   "http://purl.org/dc/terms/" : "dcterms"
                   }

# Mismo caso que el anterior, pero añadimos pares prefijo-espacio de nombres para obtener un resultado más "estético"

shaper = Shaper(graph_file_input=rdf_local_path,
                input_format=TURTLE_ITER,
                all_classes_mode=True,
                disable_comments=True,
                namespaces_dict=namespaces_dict) # Añadiendo este parámetro extra, los namespaces se incluirán en el resultado

result = shaper.shex_graph(string_output=True)

print(result)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <http://schema.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX : <http://weso.es/shapes/>

:Person
{
   rdf:type  [schema:Person]  ;
   foaf:name  xsd:string  ;
   schema:birthDate  xsd:date  ;
   schema:nationality  xsd:string  ;
   schema:knows  @:Person  ?
}


:Genre
{
   rdf:type  [schema:Genre]  ;
   schema:name  xsd:string  
}


:Book
{
   rdf:type  [schema:Book]  ;
   dcterms:title  xsd:string  ;
   schema:author  @:Person  ;
   schema:datePublished  xsd:date  ;
   schema:genre  @:Genre  +;
   schema:ISBN  xsd:string  ;
   schema:publisher  @:Organization  ;
   schema:translatedInto  xsd:string  ?
}


:Organization
{
   rdf:type  [schema:Organization]  ;
   schema:name  xsd:string  
}





In [14]:
from shexer.shaper import Shaper
from shexer.consts import TURTLE_ITER

rdf_local_path = "/content/miw_websem2425/lab_sessions/books_data.ttl"

namespaces_dict = {"http://www.w3.org/1999/02/22-rdf-syntax-ns#": "rdf",
                   "http://example.org/": "ex",
                   "http://www.w3.org/2001/XMLSchema#": "xsd",
                   "http://schema.org/" : "schema",
                   "http://xmlns.com/foaf/0.1/" : "foaf",
                   "http://purl.org/dc/terms/" : "dcterms"
                   }

# Mismo caso que el anterior, pero con estadísticas en el resultado. Esto nos permite saber datos sobre frecuencia que se observó
# cierto hecho o pistas de constraints que no llegaron al esquema final porque no conformaban con todos los nodos posibles.

shaper = Shaper(graph_file_input=rdf_local_path,
                input_format=TURTLE_ITER,
                all_classes_mode=True,
                disable_comments=False,  # Con esto a False (o no poniendo este parámetro) aparecen las estadísticas
                namespaces_dict=namespaces_dict)

result = shaper.shex_graph(string_output=True)

print(result)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <http://schema.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX : <http://weso.es/shapes/>

:Person
{
   rdf:type  [schema:Person]  ;                                # 100.0 %
   foaf:name  xsd:string  ;                                    # 100.0 %
   schema:birthDate  xsd:date  ;                               # 100.0 %
   schema:nationality  xsd:string  ;                           # 100.0 %
   schema:knows  @:Person  ?
            # 66.66666666666666 % obj: @:Person. Cardinality: {1}
}


:Genre
{
   rdf:type  [schema:Genre]  ;                                 # 100.0 %
   schema:name  xsd:string                                     # 100.0 %
}


:Book
{
   rdf:type  [schema:Book]  ;                                  # 100.0 %
   dcterms:title  xsd:string  ;                                

In [16]:
from shexer.shaper import Shaper
from shexer.consts import TURTLE_ITER

rdf_local_path = "/content/miw_websem2425/lab_sessions/books_data.ttl"

namespaces_dict = {"http://www.w3.org/1999/02/22-rdf-syntax-ns#": "rdf",
                   "http://example.org/": "ex",
                   "http://www.w3.org/2001/XMLSchema#": "xsd",
                   "http://schema.org/" : "schema",
                   "http://xmlns.com/foaf/0.1/" : "foaf",
                   "http://purl.org/dc/terms/" : "dcterms"
                   }

# Mismo caso que el anterior, pero ahora sólo se incluirán en el resultado características observadas en al menos
# el 80% de los nodos usados como ejemplo para extraer una shape. Esto afecta a las contraints con una cardinalidad
# mñinima de cero, como la cardinalidad opcional (?) o el cierre de Kleene (*).

shaper = Shaper(graph_file_input=rdf_local_path,
                input_format=TURTLE_ITER,
                all_classes_mode=True,
                disable_comments=False,
                namespaces_dict=namespaces_dict)

result = shaper.shex_graph(string_output=True,
                           acceptance_threshold=0.8) # Esta cifra es un ratio que me indica la frecuencia mínimma
                                                     # con la que se debe observar algo para incluir ese algo en el resultado
                                                     # con una cardinalidad mínima de cero.

print(result)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <http://schema.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX : <http://weso.es/shapes/>

:Person
{
   rdf:type  [schema:Person]  ;                                # 100.0 %
   foaf:name  xsd:string  ;                                    # 100.0 %
   schema:birthDate  xsd:date  ;                               # 100.0 %
   schema:nationality  xsd:string                              # 100.0 %
}


:Genre
{
   rdf:type  [schema:Genre]  ;                                 # 100.0 %
   schema:name  xsd:string                                     # 100.0 %
}


:Book
{
   rdf:type  [schema:Book]  ;                                  # 100.0 %
   dcterms:title  xsd:string  ;                                # 100.0 %
   schema:author  @:Person  ;                                  # 100.0 %
   schema:da

In [17]:
from shexer.shaper import Shaper
from shexer.consts import TURTLE_ITER

rdf_local_path = "/content/miw_websem2425/lab_sessions/books_data.ttl"

namespaces_dict = {"http://www.w3.org/1999/02/22-rdf-syntax-ns#": "rdf",
                   "http://example.org/": "ex",
                   "http://www.w3.org/2001/XMLSchema#": "xsd",
                   "http://schema.org/" : "schema",
                   "http://xmlns.com/foaf/0.1/" : "foaf",
                   "http://purl.org/dc/terms/" : "dcterms"
                   }

# Mismo caso que el anterior, pero sin ratio de aceptación (por defecto, el ratio es cero). Además, ya no pedimos
# sacar una shape para cada clase, sino sólo para un grupo de clases.

shaper = Shaper(graph_file_input=rdf_local_path,
                input_format=TURTLE_ITER,
                target_classes=["schema:Book", "schema:Genre"],
                disable_comments=False,
                namespaces_dict=namespaces_dict)

result = shaper.shex_graph(string_output=True)

print(result)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <http://schema.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX : <http://weso.es/shapes/>

:Book
{
   rdf:type  [schema:Book]  ;                                  # 100.0 %
   dcterms:title  xsd:string  ;                                # 100.0 %
   schema:author  IRI  ;                                       # 100.0 %
   schema:datePublished  xsd:date  ;                           # 100.0 %
   schema:genre  @:Genre  +;                                   # 100.0 %
            # 80.0 % obj: @:Genre. Cardinality: {1}
            # 20.0 % obj: @:Genre. Cardinality: {2}
   schema:ISBN  xsd:string  ;                                  # 100.0 %
   schema:publisher  IRI  ;                                    # 100.0 %
   schema:translatedInto  xsd:string  ?
            # 60.0 % obj: xsd:string. Car

In [26]:
from shexer.shaper import Shaper
from shexer.consts import TURTLE_ITER



namespaces_dict = {"http://www.w3.org/1999/02/22-rdf-syntax-ns#": "rdf",
                   "http://example.org/": "ex",
                   "http://www.w3.org/2001/XMLSchema#": "xsd",
                   "http://schema.org/" : "schema",
                   "http://xmlns.com/foaf/0.1/" : "foaf",
                   "http://purl.org/dc/terms/" : "dcterms",
                   "http://dbpedia.org/ontology/" : "dbo",
                   "http://dbpedia.org/resource/" : "dbr",
                   "http://dbpedia.org/property/" : "dbp",
                   "http://www.w3.org/2000/01/rdf-schema#" : "rdfs"
                   }

# En esta ocasión se mina contenido del endpoint de DBpedia. Sólo se busca sacar la shape de libro,
# sólo se usaran 10 instancias de ejemplo de libro y, tomando como raíz esas 10 instancias, sólo se traerá
# contenido del grafo a un nivel de distancia de 2 de los nodos raíz.

# PRECAUCIÓN AL EXTRAER SHAPES DE ENDPOINTS! Este proceso puede requerir hacer muchas ocnsultas SPARQL, y esto podría
# hacer que el proceso tarde mucho o, si se están usando endpoints públicos de terceros, que el proceso caiga debido a timeouts
# o baneos de IP por uso masivo del endpoint.

shaper = Shaper(target_classes=["http://dbpedia.org/ontology/Book"],
                url_endpoint="https://dbpedia.org/sparql",
                namespaces_dict=namespaces_dict,
                instances_cap=10,  # Sólamente se buscarán 10 ejemplos de instancia para extraer una shape.
                depth_for_building_subgraph=2,  # Partimos de als entidades de ejemplo y exploramos el grafo alrededor hasta esta distancia.
                track_classes_for_entities_at_last_depth_level=True) # Nos saltaremos la restricción de nivel de profundidad 2 SÓLO para tripletas de tipado

                # Aunque en el shape map sólo tengamos una clase objetivo, generaremos clases para el resto de elementos que hayan
                                        # aparecido al traer parte del grafo consumiendo en endpoint.

str_result = shaper.shex_graph(string_output=True,
                               acceptance_threshold=0.2) # Excluimos cosas que no se observen al menos el 20% de las veces



print(result)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <http://schema.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://weso.es/shapes/>

:Book
{
   rdf:type  [dbo:Book]  ;                                     # 100.0 %
   rdfs:label  xsd:string  +;                                  # 100.0 %
            # 60.0 % obj: xsd:string. Cardinality: {1}
            # 10.0 % obj: xsd:string. Cardinality: {16}
            # 10.0 % obj: xsd:string. Cardinality: {3}
            # 10.0 % obj: xsd:string. Cardinality: {5}
            # 10.0 % obj: xsd:string. Cardinality: {6}
   dbo:wikiPageID  xsd:integer  ;                              # 100.0 %
   dbo:wikiPageRevi

In [32]:
from shexer.shaper import Shaper
from shexer.consts import TURTLE_ITER



namespaces_dict = {"http://www.w3.org/1999/02/22-rdf-syntax-ns#": "rdf",
                   "http://example.org/": "ex",
                   "http://www.w3.org/2001/XMLSchema#": "xsd",
                   "http://schema.org/" : "schema",
                   "http://xmlns.com/foaf/0.1/" : "foaf",
                   "http://purl.org/dc/terms/" : "dcterms",
                   "http://dbpedia.org/ontology/" : "dbo",
                   "http://dbpedia.org/resource/" : "dbr",
                   "http://dbpedia.org/property/" : "dbp",
                   "http://www.w3.org/2000/01/rdf-schema#" : "rdfs"
                   }

# Lo mismo, pero usando un shape map en lugar de listas de clases objetivo.

# No usaré una expresión FOCUS estandar, porque podría arrojarme una consulta que tarde demasiado. Usaré una consulta SPARQL.
# Esto no forma parte del estandar de los shape maps, aunque hubo proyecto de que suese así.
# Si quieres utilizar consultas SPARQL para seleccionar nodos, sigue el patrón del ejemplo
# SPARQL' ... AQUÍ TU CONSULTA SPARQL...'@<Uri de la shape>

shape_map_raw = "SPARQL'select ?s where {?s a <http://dbpedia.org/ontology/Book>} LIMIT 100'@:Book"

shaper = Shaper(url_endpoint="https://dbpedia.org/sparql",
                shape_map_raw=shape_map_raw,
                namespaces_dict=namespaces_dict,
                instances_cap=10,
                depth_for_building_subgraph=1,
                track_classes_for_entities_at_last_depth_level=True)



str_result = shaper.shex_graph(string_output=True,
                               acceptance_threshold=0.2)



print(result)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <http://schema.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://weso.es/shapes/>

:Book
{
   rdf:type  [dbo:Book]  ;                                     # 100.0 %
   rdfs:label  xsd:string  +;                                  # 100.0 %
            # 60.0 % obj: xsd:string. Cardinality: {1}
            # 10.0 % obj: xsd:string. Cardinality: {16}
            # 10.0 % obj: xsd:string. Cardinality: {3}
            # 10.0 % obj: xsd:string. Cardinality: {5}
            # 10.0 % obj: xsd:string. Cardinality: {6}
   dbo:wikiPageID  xsd:integer  ;                              # 100.0 %
   dbo:wikiPageRevi